In [2]:
import pandas as pd

# my file structure:
# wisd hackathon 2023
# - /games (downloaded games from aws)
# -- /gameid
# - /metadata (downloaded metadata from aws)
# - this notebook
# i hope i wrote this in a way that makes sense!

In [7]:
df = pd.read_csv('joint.csv', index_col=0) # read csv generated by get_pyball.ipynb, use first col as index
df.head()

,gameId,eventType,shotClock,gameClock,wallClock,period,homePlayers,awayPlayers,playerId,pbpId,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,TOUCH,24.0,719.76,1652834779899,1,"['82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b', '267d...","['297a0fda-0a03-4003-bf39-aa92b7a730ff', '986b...",561048bb-c412-4d8b-a0bc-27d250a1a431,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,DRIBBLE,24.0,719.76,1652834779899,1,"['82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b', '267d...","['297a0fda-0a03-4003-bf39-aa92b7a730ff', '986b...",561048bb-c412-4d8b-a0bc-27d250a1a431,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,PASS,24.0,719.04,1652834780619,1,"['82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b', '267d...","['297a0fda-0a03-4003-bf39-aa92b7a730ff', '986b...",561048bb-c412-4d8b-a0bc-27d250a1a431,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,TOUCH,23.9,718.96,1652834780699,1,"['82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b', '267d...","['297a0fda-0a03-4003-bf39-aa92b7a730ff', '986b...",297a0fda-0a03-4003-bf39-aa92b7a730ff,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,DRIBBLE,23.2,718.28,1652834781379,1,"['82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b', '267d...","['297a0fda-0a03-4003-bf39-aa92b7a730ff', '986b...",297a0fda-0a03-4003-bf39-aa92b7a730ff,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
shot_df = df[df['eventType'] =="SHOT"]
print(shot_df.to_string())

                                    gameId eventType  shotClock  gameClock      wallClock  period                                                                                                                                                                                               homePlayers                                                                                                                                                                                               awayPlayers                              playerId  pbpId     GAME_ID  EVENTNUM  EVENTMSGTYPE  EVENTMSGACTIONTYPE  PERIOD WCTIMESTRING PCTIMESTRING                                                    HOMEDESCRIPTION NEUTRALDESCRIPTION                                               VISITORDESCRIPTION      SCORE SCOREMARGIN  PERSON1TYPE  PLAYER1_ID         PLAYER1_NAME  PLAYER1_TEAM_ID PLAYER1_TEAM_CITY PLAYER1_TEAM_NICKNAME PLAYER1_TEAM_ABBREVIATION  PERSON2TYPE  PLAYER2_ID         PLAYER2_NAME  PLAYER2_TEAM_